## Test CRIM intervals based on Richard's Notebooks

My test interval notebook based on richard's notebook + some tests on new methods 

In [1]:
# import items from intervals and 
import re
# from crim_intervals import *
import pandas as pd
import visualizations as viz
%cd ../intervals
from main import *


/Users/dangtrang/OneDrive - brynmawr.edu/summer 2021/crim_intervals/intervals


## Load MEI Files from CRIM or Github by pasting one or more of [these links](https://docs.google.com/spreadsheets/d/1TzRqnzgcYYuQqZR78c5nizIsBWp4pnblm2wbU03uuSQ/edit?auth_email=rfreedma@haverford.edu#gid=0) below.

*Note:  each file must be in quotation marks and separated by commas


In [2]:
corpus = CorpusBase(['https://raw.githubusercontent.com/CRIM-Project/CRIM-online/master/crim/static/mei/MEI_3.0/CRIM_Model_0001.mei'])
# corpus = CorpusBase(['https://raw.githubusercontent.com/RichardFreedman/CRIM_additional_works/main/CRIM_Ave_Test.mei_msg.mei'])

Requesting file from https://raw.githubusercontent.com/CRIM-Project/CRIM-online/master/crim/static/mei/MEI_3.0/CRIM_Model_0001.mei...
Successfully imported.


## Give the scores short names, in order according to the way they were listed above
* if more than one piece, then `piece1, piece2 = corpus.scores` 
** of course this can be `model`, `mass`, or any other name you like, as long as these are used in the requests for particular patterns below
* if just one piece, then `model = corpus.scores[0]`


In [3]:
# model, mass = corpus.scores
model = corpus.scores[0]


## Or batch process a series of files via the list below
* Be sure to choose a directory as part of `a.to_csv(f"{short_title}.csv")`
* You can use _any_ of the methods in the batch process below, so getNotes_Rests, etc.

## Now apply various methods to the scores:
* **getNoteRest** returns all the notes and rests, each voice as a column
* **getDuration** returns the durations for all notes and rests, as above
* **getMelodic** returns the melodic intervals in each voice as a column
* **getHarmonic** returns pairs of harmonic intervals between each pair of voices
* **getNgrams**  returns segments of various kinds, melodic (one voice) or modular (pairs of voices, including vertical and horizontal motion)
---

### Pandas Tools:
* **df.value_counts()**  returns summary for each pitch, duration for any type
* save results as variable, then:**`.apply(pd.Series.value_counts).fillna(0).astype(int) `**
---

### Documentation available via this command:
* for any method, use the following read documentation:
`print(model.getNgrams.__doc__)`

---


In [6]:
print(model.getNgrams.__doc__)

 Group sequences of observations in a sliding window "n" events long
        (default n=3). These cells of the resulting DataFrame can be grouped as 
        desired by setting `cell_type` to `tuple` (default), `list`, or `str`. 
        If the `exclude` parameter is passed, if any item in that list is found 
        in an ngram, that ngram will be removed from the resulting DataFrame. 
        Since `exclude` defaults to `['Rest']`, pass an empty list if you want 
        to allow rests in your ngrams.

        There are two primary modes for the `how` parameter. When set to
        "columnwise" (default), this is the simple case where the events in each
        column of the `df` DataFrame has its events grouped at the offset of the
        first event in the window. For example, to get 4-grams of melodic
        intervals:

        ip = ImportedPiece('path_to_piece')
        ngrams = ip.getNgrams(df=ip.getMelodic(), n=4)

        If `how` is set to 'modules' this will return contrap

## Methods and Parameters
within parentheses, specify paramenters.  These are optional
* kind="d" for diatonic with major(or q(uality), z(ero based), c(chromatic)
* directed=True or False
* compound=True or False
* unit=2 (or whatever increment is preferred; 1=quarter note)

In [7]:
d = model.getMelodic(kind="d", directed=True, compound=False)
d.head()

,Superius,Contratenor,PrimusTenor,SecundusTenor,Bassus
0.0,NaN,Rest,Rest,Rest,Rest
4.0,5,NaN,NaN,NaN,NaN
8.0,NaN,Rest,Rest,Rest,Rest
10.0,-2,NaN,NaN,NaN,NaN
12.0,2,NaN,NaN,NaN,NaN


# Notes and Rests


In [8]:
notes = model.getNoteRest()
notes.fillna(value= "-", inplace=True)
notes.reset_index()

,index,Superius,Contratenor,PrimusTenor,SecundusTenor,Bassus
0,0.0,G4,Rest,Rest,Rest,Rest
1,4.0,D5,-,-,-,-
2,8.0,-,Rest,Rest,Rest,Rest
3,10.0,C5,-,-,-,-
4,12.0,D5,D4,-,-,-
...,...,...,...,...,...,...
777,1065.0,-,-,-,-,F3
778,1066.0,-,B-3,-,D4,G3
779,1068.0,-,E4,-,C4,E3
780,1070.0,-,-,-,-,C3


In [9]:
notes.value_counts()
# notes.stack().value_counts()


Superius  Contratenor  PrimusTenor  SecundusTenor  Bassus
-         G4           -            -              -         20
          E4           -            -              -         15
C5        -            -            -              -         14
-         F4           -            -              -         14
D5        -            -            -              -         11
                                                             ..
-         Rest         -            G3             E3         1
                                    B-3            G3         1
                                    -              -          1
          G4           Rest         B-3            G3         1
Rest      Rest         G3           -              -          1
Length: 530, dtype: int64

In [10]:
df = notes.apply(pd.Series.value_counts).fillna(0).astype(int)
df


,Superius,Contratenor,PrimusTenor,SecundusTenor,Bassus
-,380,324,384,381,415
A3,0,4,46,41,37
A4,64,61,0,1,0
B-2,0,0,0,0,1
B-3,0,11,46,52,27
B-4,59,15,0,0,0
B3,0,1,1,2,0
B4,1,0,0,0,0
C3,0,0,0,0,20
C4,0,18,64,55,21


# Melodic Intervals
* kind='d' for diatonic; 's' for chromatic/semitone
* To save as CSV:  
`mel_int.to_csv('file_name.csv')`


In [11]:
mel_int = model.getMelodic(kind='d')
mel_int.fillna(value= "-", inplace=True)
mel_int.reset_index()


,index,Superius,Contratenor,PrimusTenor,SecundusTenor,Bassus
0,0.0,-,Rest,Rest,Rest,Rest
1,4.0,5,-,-,-,-
2,8.0,-,Rest,Rest,Rest,Rest
3,10.0,-2,-,-,-,-
4,12.0,2,-,-,-,-
...,...,...,...,...,...,...
776,1065.0,-,-,-,-,2
777,1066.0,-,-2,-,-2,2
778,1068.0,-,4,-,-2,-3
779,1070.0,-,-,-,-,-3


# Durations


In [12]:
durs = model.getDuration()
durs.fillna(value= "-", inplace=True)

## Combine Notes and Durations as One DataFrame

In [13]:
notes_durs = pd.concat([notes, durs], axis=1)
notes_durs


,Superius,Contratenor,PrimusTenor,SecundusTenor,Bassus,Superius,Contratenor,PrimusTenor,SecundusTenor,Bassus
0.0,G4,Rest,Rest,Rest,Rest,4.0,8.0,8.0,8.0,8.0
4.0,D5,-,-,-,-,6.0,-,-,-,-
8.0,-,Rest,Rest,Rest,Rest,-,4.0,8.0,8.0,8.0
10.0,C5,-,-,-,-,2.0,-,-,-,-
12.0,D5,D4,-,-,-,2.0,4.0,-,-,-
...,...,...,...,...,...,...,...,...,...,...
1065.0,-,-,-,-,F3,-,-,-,-,1.0
1066.0,-,B-3,-,D4,G3,-,2.0,-,2.0,2.0
1068.0,-,E4,-,C4,E3,-,4.0,-,4.0,2.0
1070.0,-,-,-,-,C3,-,-,-,-,2.0


# Select Columns for One Voice


In [14]:
notes_durs_s = notes_durs.iloc[:, [0,6]]
notes_durs_s

,Superius,Contratenor
0.0,G4,8.0
4.0,D5,-
8.0,-,4.0
10.0,C5,-
12.0,D5,4.0
...,...,...
1065.0,-,-
1066.0,-,2.0
1068.0,-,4.0
1070.0,-,-


## N-Grams in Each Voice
* for Melodic or Durations

In [1]:
corpus = CorpusBase(['https://crimproject.org/mei/CRIM_Model_0017.mei'])
model = corpus.scores[0]
mel = model.getMelodic(kind='d', compound=False, unit=4)
mel_ngrams = model.getNgrams(df=mel, n=4, cell_type=str)
mel_ngrams



NameError: name 'CorpusBase' is not defined

### Plot visualizations for all patterns in Ngrams

The `plot_ngrams_heatmap(model, ngrams_df)` method from visualizations plot a heat map based on a model and an ngram dataframe. 

We could retrieve a chart and the dataframe the chart was plotted using:

```
chart, chart_df = viz.plot_ngrams_heatmap(model, mel_ngrams)
```

Or if we don't care about the dataframe, we could display the plot with: 
```
viz.plot_ngrams_heatmap(model, mel_ngrams)[0]
```
The `[0]` means that we are displaying the first of the two outputs from the method.

In [16]:
chart, chart_df = viz.plot_ngrams_heatmap(model, mel_ngrams)

In [17]:
chart_df

,start,voices,pattern,end
0,4.0,[Discantus],"1, 1, 1, 2",5.0
1,8.0,[Discantus],"1, 1, 2, 1",9.0
2,12.0,[Discantus],"1, 2, 1, 1",13.0
3,16.0,[Discantus],"2, 1, 1, 1",17.0
4,20.0,[Discantus],"1, 1, 1, 2",21.0
...,...,...,...,...
1975,1480.0,[Bassus],"4, 1, 1, 1",1481.0
1976,1484.0,[Bassus],"1, 1, 1, 1",1485.0
1977,1488.0,[Bassus],"1, 1, 1, 1",1489.0
1978,1492.0,[Bassus],"1, 1, 1, 1",1493.0


In [2]:
chart

NameError: name 'chart' is not defined

In [19]:
out2 = mel_ngrams[mel_ngrams == '1, 2, 1, 2'].stack().dropna().to_frame()
# out2.reset_index(inplace=True)
# out2["measure"] = out2['level_0']/8+1
# out2
out2.index.to_list()


[(300.0, '[Contratenor]'),
 (300.0, '[QuintaVox]'),
 (304.0, '[Discantus]'),
 (328.0, '[Tenor]')]

Plot visualizations for specific patterns from ngrams

In [20]:
viz.plot_ngrams_heatmap(model, mel_ngrams, patterns=['1, 1, 1, 1'])[0]

alt.VConcatChart(...)

# Harmonic Intervals between Voices

In [23]:
# har = model.getHarmonic()
# regHar = model.regularize(df=har, unit=2)
# regHar

In [24]:
harm = model.getHarmonic(kind="d", compound=True)
h_ng = model.getNgrams(df=harm, how='modules', exclude=['Rest'], cell_type="str")
h_ng.stack().value_counts().to_frame().head(50)


,0
"3_Held, 4_Held, 5",16
"10_2, 9_2, 8",13
"8_2, 7_2, 6",12
"6_-2, 7_Held, 6",12
"1_Held, 2_Held, 3",12
"7_Held, 6_Held, 6",11
"3_2, 2_2, 1",11
"3_-2, 3_-2, 3",10
"10_Held, 11_Held, 12",10
"8_2, 7_Held, 6",9


In [25]:
h_ng[h_ng == '-2_-3, 3_Held, 4'].stack().dropna().to_frame()

,,0
123.0,[Bassus]_[Contratenor],"-2_-3, 3_Held, 4"
1435.0,[Tenor]_[Contratenor],"-2_-3, 3_Held, 4"
537.0,[QuintaVox]_[Contratenor],"-2_-3, 3_Held, 4"


Plot visualizations for harmonic intervals between voices

In [26]:
h_ng[h_ng == '-3_Held, -2_4, -4'].stack().dropna().to_frame()

,,0
1688/3,[QuintaVox]_[Contratenor],"-3_Held, -2_4, -4"


In [27]:
chart, chart_df = viz.plot_ngrams_heatmap(model, h_ng, patterns=['-2_-3, 3_Held, 4', '-3_Held, -2_4, -4'], voices=['[QuintaVox]_[Contratenor]'])
chart

alt.VConcatChart(...)

In [ ]:
chart_df

In [ ]:
out3 = h_ng[h_ng == '7_-, 6_-2, 8'].stack().dropna().to_frame()
out3.reset_index(inplace=True)
out3["measure"] = out3['level_0']/8+1
out3

# Two-Voice Modules as N-Grams

* 'modules' is in fact the default
* 'unit' refers to the durational increment


In [ ]:
ng = model.getNgrams(df=harm, how='columnwise', exclude=['Rest'])
ng.head()
ng.stack().to_frame()

In [ ]:
viz.plot_ngrams_heatmap(model, ng)[0]

In [ ]:
ng2 = ng[ng.apply(lambda row: row.astype(str).str.contains('7, -, 6, -2, 8').any(), axis=1)].copy()
# find_soggetto.reset_index(inplace=True)
# find_soggetto["measure"] = find_soggetto['index']/8+1
ng2.head(50)

# Filter by Any String of Intervals

In [ ]:


filtered = ng[ng.apply(lambda row: row.astype(str).str.contains('7').any(), axis=1)].copy()
# filtered = ng[ng.apply(lambda row: row.astype(str).str.contains('6_-2, 6_-2, 6').any(), axis=1)].copy()

filtered.reset_index(inplace=True)
filtered["measure"] = filtered['index']/8+1
filtered



## Ngrams with Real Durations

In [ ]:
print(model.getNgrams.__doc__)

In [ ]:
ng = model.getNgrams(how='modules', cell_type=str)
ng


In [ ]:
filtered = ng[ng.apply(lambda row: row.astype(str).str.contains('6_-2, 8').any(), axis=1)].copy()
filtered.reset_index(inplace=True)
# filtered["measure"] = filtered['index']/8+1
filtered


In [ ]:
df = ng.value_counts()
df = ng.stack().value_counts()
df.tail(50)

In [ ]:
corpus = CorpusBase(['https://crimproject.org/mei/CRIM_Mass_0015_2.mei'])
model = corpus.scores[0]
mel = model.getMelodic(kind='d')
mel_ngrams = model.getNgrams(df=mel, n=4, cell_type=str)


In [ ]:
viz.plot_ngrams_heatmap(model, mel_ngrams)[0]

In [ ]:
viz.plot_ngrams_heatmap(model, mel_ngrams, patterns=['-2, -2, -2, -2'])[0]

In [ ]:
out = mel_ngrams[mel_ngrams == '-2, -3, 2, -2'].stack().dropna()
out
# out.reset_index(inplace=True)
# out["measure"] = out['index']/8+1

In [ ]:
corpus = CorpusBase(['https://crimproject.org/mei/CRIM_Model_0017.mei'])
model = corpus.scores[0]
har = model.getHarmonic(kind='d')
har_ngrams = model.getNgrams(how='modules', df=har, n=3, cell_type=str)
# har_ngrams.to_csv('model_17_harmonic')

har_ngrams[har_ngrams == '3_Held, 1_-2, 3'].stack().dropna()



## Test models methods

Here I create test for the new methods I developed

In [ ]:
files = [title.split("/")[-1] for title in titles]

In [ ]:
prefix = "https://raw.githubusercontent.com/CRIM-Project/CRIM-online/master/crim/static/mei/MEI_3.0/"
for file in files:
    corpus = CorpusBase([prefix + file])
    model = corpus.scores[0]
    print(model.getSoundingCount().values)